# پیاده‌سازی سیستم خبره حیوانات

یک مثال از [دوره مقدماتی هوش مصنوعی](http://github.com/microsoft/ai-for-beginners).

در این نمونه، ما یک سیستم ساده مبتنی بر دانش را برای تعیین یک حیوان بر اساس برخی ویژگی‌های فیزیکی پیاده‌سازی خواهیم کرد. سیستم می‌تواند توسط درخت AND-OR زیر نمایش داده شود (این بخشی از کل درخت است، ما می‌توانیم به آسانی قوانین بیشتری اضافه کنیم):

![](../../../../../../translated_images/fa/AND-OR-Tree.5592d2c70187f283.webp)


## پوسته سیستم‌های خبره خودمان با استنتاج معکوس

بیایید سعی کنیم یک زبان ساده برای نمایش دانش مبتنی بر قواعد تولید تعریف کنیم. از کلاس‌های پایتون به عنوان کلیدواژه برای تعریف قواعد استفاده خواهیم کرد. اساساً سه نوع کلاس وجود خواهد داشت:
* `Ask` نمایانگر سوالی است که باید از کاربر پرسیده شود. این کلاس مجموعه‌ای از پاسخ‌های ممکن را در بر دارد.
* `If` نمایانگر یک قاعده است و فقط یک شکرنگار نحوی برای ذخیره محتوای قاعده می‌باشد.
* `AND`/`OR` کلاس‌هایی برای نمایندگی شاخه‌های AND/OR در درخت هستند. آنها فقط لیست آرگومان‌های درونشان را ذخیره می‌کنند. برای ساده‌سازی کد، تمام عملکردها در کلاس والد `Content` تعریف شده‌اند.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

در سیستم ما، حافظه کاری شامل لیستی از **حقایق** به صورت **جفت‌های ویژگی-مقدار** خواهد بود. پایگاه دانش را می‌توان به عنوان یک فرهنگ لغت بزرگ تعریف کرد که اعمال (حقایق جدیدی که باید در حافظه کاری وارد شوند) را به شرایطی که به صورت عبارات AND-OR بیان شده‌اند، نگاشت می‌کند. همچنین، برخی از حقایق می‌توانند `Ask` شوند.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

برای انجام استنتاج معکوس، کلاس `Knowledgebase` را تعریف خواهیم کرد. این کلاس شامل خواهد بود از:
* حافظه کاری `memory` - یک دیکشنری که ویژگی‌ها را به مقادیر نگاشت می‌کند
* قوانین پایگاه دانش `rules` به قالبی که در بالا تعریف شده است

دو روش اصلی عبارتند از:
* `get` برای به‌دست آوردن مقدار یک ویژگی، انجام استنتاج در صورت نیاز. برای مثال، `get('color')` مقدار یک جایگاه رنگ را به دست می‌آورد (در صورت نیاز سوال می‌پرسد و مقدار را برای استفاده‌های بعدی در حافظه کاری ذخیره می‌کند). اگر `get('color:blue')` را بپرسیم، برای رنگ سوال می‌پرسد و سپس مقدار `y`/`n` را بسته به رنگ برمی‌گرداند.
* `eval` استنتاج واقعی را انجام می‌دهد، یعنی درخت AND/OR را مرور می‌کند، زیرهدف‌ها را ارزیابی می‌کند و غیره.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

حال بیایید پایگاه دانش حیوانات خود را تعریف کنیم و مشاوره را انجام دهیم. توجه داشته باشید که این فراخوانی از شما سوال خواهد پرسید. شما می‌توانید با تایپ `y`/`n` برای سوالات بلی-خیر پاسخ دهید، یا با مشخص کردن عدد (۰..N) برای سوالات با پاسخ‌های چندگزینه‌ای طولانی‌تر پاسخ دهید.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## استفاده از Experta برای استنتاج پیشرو

در مثال بعدی، سعی خواهیم کرد استنتاج پیشرو را با استفاده از یکی از کتابخانه‌های نمایش دانش، [Experta](https://github.com/nilp0inter/experta) پیاده‌سازی کنیم. **Experta** یک کتابخانه برای ایجاد سیستم‌های استنتاج پیشرو در پایتون است که طراحی شده تا مشابه سیستم کلاسیک قدیمی [CLIPS](http://www.clipsrules.net/index.html) باشد.

همچنین می‌توانستیم استنتاج زنجیره‌ای پیشرو را خودمان بدون مشکلات زیاد پیاده‌سازی کنیم، اما پیاده‌سازی‌های ساده معمولاً چندان بهینه نیستند. برای تطبیق موثرتر قوانین از الگوریتم ویژه‌ای به نام [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) استفاده می‌شود.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

ما سیستم خود را به صورت یک کلاس که زیرکلاسی از `KnowledgeEngine` است تعریف می‌کنیم. هر قاعده توسط یک تابع جداگانه با تزئین‌کننده `@Rule` تعریف می‌شود، که مشخص می‌کند قاعده کی باید فعال شود. درون قاعده، می‌توانیم حقایق جدید را با استفاده از تابع `declare` اضافه کنیم، و افزودن این حقایق منجر به فراخوانی برخی قواعد بیشتر توسط موتور استنتاج پیشرو خواهد شد.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

زمانی که یک پایگاه دانش تعریف کردیم، حافظه کاری خود را با چند حقیقت اولیه پر می‌کنیم، و سپس متد `run()` را برای انجام استنتاج فراخوانی می‌کنیم. شما می‌توانید ببینید که در نتیجه حقایق استنتاج شده جدید به حافظه کاری اضافه می‌شوند، از جمله حقیقت نهایی درباره حیوان (اگر تمام حقایق اولیه را به درستی تنظیم کرده باشیم).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از خدمات ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. با اینکه تلاش ما برای دقت است، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌هایی باشند. سند اصلی به زبان بومی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچگونه سوءتفاهم یا برداشت نادرستی که از استفاده این ترجمه ناشی شود، نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
